1. Data Cleaning  
    1.1 Gain basic CSV overview  
    1.2 Formatiing NaN values    
    1.3 Transforming Coordinates  
    1.4 Transforming Timestamps  
    1.5 Checking for duplicates  
  
2. Data Overview  
    2.1 Gain Overview of data  
    2.2 Plot lines




1.1 Gain basic CSV overview

In [1]:
# import basic libs
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import shapely

In [2]:
# import csv as pd
data = pd.read_csv("data/atlantic.csv")

In [3]:
""" Plotting column an row aounts & header"""
print("Pandas Dataframe Info:\n")
data.info()
print("\n\nPandas DataFrame Head:\n")
data.head()

Pandas Dataframe Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49105 entries, 0 to 49104
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                49105 non-null  object
 1   Name              49105 non-null  object
 2   Date              49105 non-null  int64 
 3   Time              49105 non-null  int64 
 4   Event             49105 non-null  object
 5   Status            49105 non-null  object
 6   Latitude          49105 non-null  object
 7   Longitude         49105 non-null  object
 8   Maximum Wind      49105 non-null  int64 
 9   Minimum Pressure  49105 non-null  int64 
 10  Low Wind NE       49105 non-null  int64 
 11  Low Wind SE       49105 non-null  int64 
 12  Low Wind SW       49105 non-null  int64 
 13  Low Wind NW       49105 non-null  int64 
 14  Moderate Wind NE  49105 non-null  int64 
 15  Moderate Wind SE  49105 non-null  int64 
 16  Moderate Wind SW  49105 non-null  

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Minimum Pressure,...,Low Wind SW,Low Wind NW,Moderate Wind NE,Moderate Wind SE,Moderate Wind SW,Moderate Wind NW,High Wind NE,High Wind SE,High Wind SW,High Wind NW
0,AL011851,UNNAMED,18510625,0,,HU,28.0N,94.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,AL011851,UNNAMED,18510625,600,,HU,28.0N,95.4W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,18510625,1200,,HU,28.0N,96.0W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,18510625,1800,,HU,28.1N,96.5W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,18510625,2100,L,HU,28.2N,96.8W,80,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


We see that 22 columns are available, containing 49104 rows with hurrican data points.  
For now, the following info is important:  
- Unique IDs
- Hurricanes ordered by 'string' names
- Date and time are in separate rows -> maybe they should be converted to datetime format
- coordinates are objects, likely shapely -> Since the bearing will be calculated manualy, the coordinates shall be converted to numbers
- the int value -999 is most likely the NaN value -> Conversion to proper numpy.nan

# 1.2 Formatiing NaN values  
Using the pd.replace() method, all numerical -999 values are replaced with the proper numpy NaN data type

In [4]:
"""convert -999 to NaN"""
data = data.replace(-999,np.NaN)

# 1.3 Transforming Coordinates  
By iterating over the latitude and longitude strings, the last index is checked. The string is then transformed into floats and based on the heading turned negative or not.

In [5]:
"""convert string coordiantes no numerical"""
# empty list to append to
lat_transformed = []
# iterate over dataframe lat
for i in data["Latitude"]:
    # initiate and clear variable to hold string first then converted float
    # tmp_val = 0
    # check if N at end
    if i[-1]=="N":
        # assign temp value, exclude last index (N), convert to float
        tmp_val = float(i[:-1])
    #check if S at the end
    if i[-1]=="S":
        # assign temp value, exclude last index (N), convert to float
        # substract double the amount to get same value in negative
        tmp_val =  float(i[:-1]) - (float(i[:-1])*2)
    # append temp value to lat list
    lat_transformed.append(tmp_val)

#empty list to append to
lon_transformed = []
# iterate over dataframe lon
for i in data["Longitude"]:
    # check if E at end
    if i[-1]=="E":
        # assign temp value, exclude last index (N), convert to float
        tmp_val = float(i[:-1])
    # check if W at the end
    if i[-1]=="W":
        # assign temp value, exclude last index (N), convert to float
        # substract double the amount to get same value in negative
        tmp_val =  float(i[:-1]) - (float(i[:-1])*2)
    # append temp value to lon list
    lon_transformed.append(tmp_val)

# update lat and lon columns with the transformed info
data["Latitude"] = lat_transformed
data["Longitude"] = lon_transformed

# 1.4 Transforming Timestamps  
Since the time and date are in different columns and not the proper data types, the two columns are iterated over and concatenated. using the datetime library, the resulting string is then turned into the datetime datatype. For that to happen, the information is unified by adding leading zeros to the hour&minute information, so that the format information can be passed to the strptime() function.

In [6]:
"""transform date & time columns to datetime format"""
import datetime as dt
# empty list that holds tiemstamps
timestamps = []
# iterate over date and time
for date,time in zip(data["Date"],data["Time"]):
    # adding leading zeroes to time until length = 4
    while len(str(time)) < 4:
        time = str(0)+str(time)
    # concatenating date and time separated by space, appending 00 for the seconds
    timestamp = str(date) + " " + str(time) + "00"
    # transform string via strptime, passing format info
    timestamps.append(dt.datetime.strptime(timestamp, "%Y%m%d %H%M%S"))
# append dataframe with timestamps list
data["Timestamp"] = timestamps
data.drop(["Date","Time"], axis = 1, inplace = True)

# 1.5 Checking for Duplicates

Since there are no unique datapoint identifiers, the only way duplicates cam be detected is by seeing if a certain combination of attributes occurs in more than one row. For that, it is checked if/how many duplicates the pd.duplicated() function finds.

In [20]:
counter_false = 0
counter_true = 0
for i in data.duplicated():
    if i == True:
        counter_true = counter_true + 1
    else:
        counter_false = counter_false + 1
print("No. of duplicates in DF:\t"+str(counter_true))
print("No. of unique rows in DF:\t"+str(counter_false))

No. of duplicates in DF:	0
No. of unique rows in DF:	49105
